<a href="https://colab.research.google.com/github/ElaheTorabi/Masters-Thesis/blob/main/Fuel_Data_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Fuel Data Preprocessing and Cleaning**

Importing necassary packages

In [ ]:
import pandas as pd
import numpy as np
from datetime import datetime
import matplotlib.pyplot as plt
from google.colab import drive

Importing the dataset

In [ ]:
df = pd.read_excel('....xlsx')
df.head()

Data Preparation

In [ ]:
# Convert date column to datetime
df['Date_Hour_Desc'] = pd.to_datetime(df['Date_Hour_Desc'], errors='coerce')

# Sort by date
df = df.sort_values('Date_Hour_Desc')

# Set timestamp as index for easier plotting
df = df.set_index('Date_Hour_Desc')

In [ ]:
# Find rows where LevelLiters is greater than TotalLiters
invalid_rows = df[df['LevelLiters'] > df['TotalLiters']]

# Print summary
print(f"Number of records where LevelLiters > TotalLiters: {len(invalid_rows)}")

# Display sample
if len(invalid_rows) > 0:
    print("\nSample invalid readings:")
    print(invalid_rows[['DEVICE_ID', 'LevelLiters', 'TotalLiters']].head(10))

# Save invalid rows separately
invalid_rows.to_excel("invalid_LevelLiters_vs_TotalLiters.xlsx", index=True)


Filtering Data

In [ ]:
df_clean = df[df['LevelLiters'] <= df['TotalLiters']].copy()

print("Original dataset size:", len(df))
print("Cleaned dataset size :", len(df_clean))
print("Rows removed:", len(df) - len(df_clean))


In [ ]:
# Bring the datetime index back as a column
df_clean = df_clean.reset_index()

# Double-check
print(df_clean.columns[:5])


Fetching hourly temperature data

In [ ]:
import requests
import pandas as pd
from tqdm import tqdm

# Ensure datetime
df_clean['Date_Hour_Desc'] = pd.to_datetime(df_clean['Date_Hour_Desc'])
df_clean = df_clean.sort_values('Date_Hour_Desc')

# Unique devices with coordinates
device_location_map = df_clean[['DEVICE_ID', 'OGI_LAT', 'OGI_LONG']].drop_duplicates()
print(f"Found {len(device_location_map)} unique devices with coordinates.")

# Determine overall time range
start_date = df_clean['Date_Hour_Desc'].min().strftime("%Y-%m-%d")
end_date = df_clean['Date_Hour_Desc'].max().strftime("%Y-%m-%d")
print(f"Fetching hourly weather between {start_date} and {end_date}")

weather_records = []

# Loop per device
for _, row in tqdm(device_location_map.iterrows(), total=len(device_location_map)):
    device_id = row['DEVICE_ID']
    lat, lon = row['OGI_LAT'], row['OGI_LONG']

    try:
        url = (
            f"https://archive-api.open-meteo.com/v1/archive?"
            f"latitude={lat}&longitude={lon}"
            f"&start_date={start_date}&end_date={end_date}"
            f"&hourly=temperature_2m&timezone=auto"
        )
        r = requests.get(url)
        d = r.json()
        if "hourly" in d and "time" in d["hourly"]:
            temp_df = pd.DataFrame({
                "datetime": pd.to_datetime(d["hourly"]["time"]),
                "temperature": d["hourly"]["temperature_2m"],
                "DEVICE_ID": device_id
            })
            weather_records.append(temp_df)
    except Exception as e:
        print(f" Failed for DEVICE_ID {device_id}: {e}")

# Combine all devices’ temperature data
df_weather_hourly = pd.concat(weather_records, ignore_index=True)
df_weather_hourly.to_excel("df_weather_hourly.xlsx", index=False)

print(f" Saved hourly weather data ({len(df_weather_hourly)} records) to df_weather_hourly.xlsx")


Merging the temperature data with fuel dataset

In [ ]:
# Convert to datetime index
df_weather_hourly['datetime'] = pd.to_datetime(df_weather_hourly['datetime'])
df_clean['Date_Hour_Desc'] = pd.to_datetime(df_clean['Date_Hour_Desc'])

# Merge on DEVICE_ID + timestamp
df_merged = pd.merge(
    df_clean,
    df_weather_hourly,
    left_on=['DEVICE_ID', 'Date_Hour_Desc'],
    right_on=['DEVICE_ID', 'datetime'],
    how='left'
)

# Drop redundant column
df_merged = df_merged.drop(columns=['datetime'])
print(f" Merged dataset shape: {df_merged.shape}")
print(df_merged[['DEVICE_ID', 'Date_Hour_Desc', 'LevelLiters', 'temperature']].head())


In [ ]:
print("Missing temperature count:", df_merged['temperature'].isna().sum())
print("NaN count in df_merged before cleaning:", df_merged['LevelLiters'].isna().sum())

Detecting refilling and noise spikes in raw fuel level data and classifying them

In [ ]:
from scipy.stats import zscore

def detect_refills_with_datetime_index_raw(
    df,
    device_id,
    z_threshold=3,
    capacity_factor_refill=0.1,
    capacity_factor_noise=0.02,
    window_after=30,          # hours forward to test persistence
    persist_points=8,
    flat_std_threshold=1,     # liters tolerance for detecting flat plateaus
    reverse_window_hours=30   # hours ahead to check for drop-back reversals
):

    # Filter one device
    df_device = df[df["DEVICE_ID"] == device_id].copy()

    # Make sure time index is correct
    if not np.issubdtype(df_device.index.dtype, np.datetime64):
        if 'Date_Hour_Desc' in df_device.columns:
            df_device['Date_Hour_Desc'] = pd.to_datetime(df_device['Date_Hour_Desc'], errors='coerce')
            df_device = df_device.set_index('Date_Hour_Desc')
        else:
            raise ValueError(" DataFrame must have a datetime index or 'Date_Hour_Desc' column.")

    df_device = df_device.sort_index()

    # Compute diffs
    df_device["diff"] = df_device["LevelLiters"].diff()

    # Z-score spike candidates
    df_device["zscore"] = zscore(df_device["diff"].fillna(0))
    candidates = df_device.index[abs(df_device["zscore"]) > z_threshold]

    print(f"Device {device_id}: Z-score spike candidates = {len(candidates)}")

    refills, noise = [], []

    # A. Handle possible flat plateau at start
    first_level = df_device.iloc[0]['LevelLiters']
    first_capacity = df_device.iloc[0]['TotalLiters']

    if first_level > 0.8 * first_capacity:
        print(" Initial high level detected at dataset start (possible calibration / noise).")

        start_flat_points = []
        for t in df_device.index:
            if abs(df_device.loc[t, "LevelLiters"] - first_level) <= flat_std_threshold:
                start_flat_points.append(t)
            else:
                break

        print(f"Initial flat plateau length: {len(start_flat_points)} points")
        noise.extend(start_flat_points)

    # B. Normal spike-based logic
    for idx in candidates:
        pos = df_device.index.get_loc(idx)
        if pos == 0:
            continue

        jump = df_device.loc[idx, "diff"]
        tank_capacity = df_device.loc[idx, "TotalLiters"]

        # Thresholds
        refill_jump_threshold = capacity_factor_refill * tank_capacity
        noise_jump_threshold  = capacity_factor_noise * tank_capacity

        prev_level = df_device.iloc[pos - 1]["LevelLiters"]

        # Persistence check
        window_end = idx + pd.Timedelta(hours=window_after)
        future = df_device.loc[idx:window_end, "LevelLiters"]

        persistent_threshold = max(0.01 * tank_capacity, 0.1 * jump)
        persistent = (future > prev_level + persistent_threshold).sum() >= persist_points

        # Detect drop-back within persistence window
        early_reversal = (future < prev_level + 0.1 * tank_capacity).any()

        # Reverse window (after persistence)
        reversal_window = df_device.loc[
            window_end : window_end + pd.Timedelta(hours=reverse_window_hours),
            "LevelLiters"
        ]
        reversed_drop = (
            (reversal_window < prev_level + 0.1 * tank_capacity).any()
            if len(reversal_window) > 0
            else False
        )

        # Detect flat plateau
        plateau_level = df_device.loc[idx, "LevelLiters"]
        after_idx = df_device.index[pos:]
        flat_points = []
        for t in after_idx:
            if abs(df_device.loc[t, "LevelLiters"] - plateau_level) <= flat_std_threshold:
                flat_points.append(t)
            else:
                break
        flat_plateau = len(flat_points) > persist_points

        # Decision
        if jump > refill_jump_threshold and persistent and not reversed_drop and not early_reversal and not flat_plateau:
            refills.append(idx)
        else:
            noise.extend(flat_points)

    # Deduplicate and sort
    noise = sorted(pd.Index(noise).unique())

    print(f"Device {device_id}: REFILLS = {len(refills)}, NOISE (incl. flats) = {len(noise)}")

    refill_df = df_device.loc[refills]
    noise_df  = df_device.loc[noise]

    # Plot
    plt.figure(figsize=(14,6))
    plt.plot(df_device.index, df_device["LevelLiters"], color="steelblue", label="Raw LevelLiters", linewidth=2)

    if len(refill_df) > 0:
        plt.scatter(refill_df.index, refill_df["LevelLiters"],
                    color="green", marker="^", s=80, label="Refill Event")
    if len(noise_df) > 0:
        plt.scatter(noise_df.index, df_device.loc[noise_df.index, "LevelLiters"],
                    color="red", marker="x", s=70, label="Noise Spike")

    plt.title(f"Refill & Noise Detection – Device {device_id}", fontsize=14)
    plt.xlabel("Timestamp")
    plt.ylabel("LevelLiters (raw)")
    plt.legend()
    plt.grid(True, linestyle="--", alpha=0.5)
    plt.tight_layout()
    plt.show()

    return refill_df, noise_df, df_device, device_id
refills, noise, processed, device_id = detect_refills_with_datetime_index_raw(df_merged, device_id=558)

Cleaning noisy spikes from LevelLiters (raw data), interpolating missing values, and resampling to hourly frequency.

In [ ]:
def clean_noise_interpolate(df_device, noise_df, device_id):

    # Copy input device data
    df_stl = df_device.copy()

    # Replace noise points with NaN
    df_stl.loc[noise_df.index, 'LevelLiters'] = np.nan

    # Interpolate missing values (time-based)
    df_stl['LevelLiters'] = df_stl['LevelLiters'].interpolate(method='time')

    print("NaN count before resampling:", df_stl['LevelLiters'].isna().sum())
    print("Number of timestamps before resampling:", len(df_stl))


    # Ensure regular hourly frequency
    df_stl = df_stl.asfreq('h')  # enforce 1-hour interval

    df_stl['LevelLiters'] = (
    df_stl['LevelLiters']
    .interpolate(method='time', limit_direction='both')
    .bfill()  # fill start
    .ffill()  # fill end
    )

    print("NaN count after resampling:", df_stl['LevelLiters'].isna().sum())
    print("Number of timestamps after resampling:", len(df_stl))


    # Visual check
    plt.figure(figsize=(14,6))
    plt.plot(df_device.index, df_device['LevelLiters'],
             color='lightgray', label='Original (with noise)', linewidth=1.5)
    plt.plot(df_stl.index, df_stl['LevelLiters'],
             color='steelblue', label='Cleaned (interpolated)', linewidth=2)
    if len(noise_df) > 0:
        plt.scatter(noise_df.index, df_device.loc[noise_df.index, 'LevelLiters'],
                    color='red', marker='x', s=60, label='Removed Noise')
    plt.legend()
    plt.title(f"Noise Removal and Interpolation – Device {device_id}")
    plt.xlabel("Timestamp")
    plt.ylabel("LevelLiters (Raw)")
    plt.grid(True, linestyle="--", alpha=0.5)
    plt.tight_layout()
    plt.show()

    # Return cleaned data
    return df_stl

# Clean noise using detected indices
df_stl = clean_noise_interpolate(processed, noise, device_id)

# Check if NaNs remain
print("Remaining NaN values per column:\n", df_stl.isna().sum())


In [ ]:
df_stl['diff'] = df_stl['diff'].fillna(0)
nan_diff_rows = df_stl[df_stl['diff'].isna()]
print(nan_diff_rows[['LevelLiters']].head(5))


Applying temperature correction

In [ ]:
# Define constants
BETA = 0.00089   # thermal expansion coefficient of diesel (1/°C)
T_REF = 15       # reference temperature (°C) — baseline at which tank was calibrated

# Compute temperature-corrected level
df_stl['LevelLiters_corrected'] = df_stl['LevelLiters'] / (1 + BETA * (df_stl['temperature'] - T_REF))

# Visualize the correction effect
import matplotlib.pyplot as plt

plt.figure(figsize=(14,6))
plt.plot(df_stl.index, df_stl['LevelLiters'], color='blue', alpha=0.6, label='Raw LevelLiters')
plt.plot(df_stl.index, df_stl['LevelLiters_corrected'], color='darkorange', label='Temperature-corrected Level')
plt.title("Temperature Correction Applied to Fuel Level")
plt.xlabel("Timestamp")
plt.ylabel("Fuel Level (Liters)")
plt.legend()
plt.grid(True, linestyle="--", alpha=0.5)
plt.tight_layout()
plt.show()

# Quick sanity check ---
print("Temperature correction applied successfully!")
print("Raw vs Corrected correlation with temperature:")
print("Raw:      ", df_stl[['LevelLiters','temperature']].corr().iloc[0,1])
print("Corrected:", df_stl[['LevelLiters_corrected','temperature']].corr().iloc[0,1])

In [ ]:
df_stl['temperature'] = df_stl['temperature'].interpolate(method='time')
df_stl['LevelLiters_corrected'] = df_stl['LevelLiters_corrected'].interpolate(method='time')
print("Any NaNs:", df_stl.isna().any().any())
print("Any duplicates:", df_stl.index.duplicated().sum())


Detecting and cleaning outliers (IQR-based)

In [ ]:
# --- Compute temperature-corrected consumption ---
df_stl['Consumption_corrected'] = -df_stl['LevelLiters_corrected'].diff()
df_stl.loc[df_stl['Consumption_corrected'] < 0, 'Consumption_corrected'] = 0

# Smoothing
df_stl['Consumption_corrected'] = (
    df_stl['Consumption_corrected']
    .rolling(window=3, center=True, min_periods=1)
    .mean()
)

# Detect and clean outliers (IQR-based)
Q1, Q3 = df_stl["Consumption_corrected"].quantile([0.25, 0.75])
IQR = Q3 - Q1

outlier_mask = (
    (df_stl["Consumption_corrected"] < Q1 - 1.5 * IQR) |
    (df_stl["Consumption_corrected"] > Q3 + 1.5 * IQR)
)

print(f"Detected {outlier_mask.sum()} outlier points — replacing with interpolated values")

# Replace outliers with NaN
df_stl.loc[outlier_mask, "Consumption_corrected"] = pd.NA

# Interpolate missing values (fill cleaned outliers smoothly)
df_stl["Consumption_corrected"] = df_stl["Consumption_corrected"].interpolate(
    method="linear", limit_direction="both"
)

# Final sanity check
nan_count = df_stl["Consumption_corrected"].isna().sum()
print(f"✅ Cleaning complete. Remaining NaN in 'Consumption': {nan_count}")


# Forcasting Models

ETS Forecasting (Hourly Fuel Consumption for the last 30 days)

In [ ]:
from statsmodels.tsa.holtwinters import ExponentialSmoothing
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score

# Select the hourly consumption series
series = df_stl["Consumption_corrected"].copy()
series = series + 1e-6  # small offset to avoid zeros

# Split into training and last 30 days (720 hours)
train = series[:-720]
test = series[-720:]

# Fit ETS model
ets_model = ExponentialSmoothing(
    train,
    trend="add",
    seasonal="add",
    seasonal_periods=24,   # 24-hour daily cycle for hourly data
    initialization_method="estimated"
).fit()

# Forecast the next 720 hours (30 days)
forecast_horizon = 720
forecast = ets_model.forecast(forecast_horizon)

# Evaluate
mae = mean_absolute_error(test, forecast)
rmse = np.sqrt(mean_squared_error(test, forecast))
nrmse = rmse / (np.max(test) - np.min(test))
mase = mae / np.mean(np.abs(np.diff(train)))
r2 = r2_score(test, forecast)



print("ETS Forecast Evaluation – Last 30 Days")
print("-------------------------------------------")
print(f"MAE  : {mae:.3f}")
print(f"RMSE : {rmse:.3f}")
print(f"NRMSE (range) : {nrmse*100:.2f}%")
print(f"R²   : {r2:.3f}")



# Plot results
plt.figure(figsize=(12,5))
plt.plot(test.index, test, label="Actual (Last 30 Days)", color="black", linewidth=1.8)
plt.plot(forecast.index, forecast, label="Predicted (ETS)", color="blue", linewidth=2)

plt.title(f"ETS Forecast (Hourly Fuel Consumption)-Device {int(device_id)}", fontsize=14)
plt.xlabel("Timestamp")
plt.ylabel("Fuel Consumption (Liters/hour)")
plt.legend()
plt.grid(True, linestyle="--", alpha=0.5)
plt.gcf().autofmt_xdate(rotation=45)   # auto-rotate to avoid overlap
plt.tight_layout()

plt.tight_layout()
plt.show()


LightGBM Forecasting

In [ ]:
import lightgbm as lgb
import seaborn as sns
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score

# Prepare base hourly data
df_hourly = df_stl[['Consumption_corrected', 'temperature']].copy()

# Small offset to avoid division errors
df_hourly['Consumption_corrected'] = df_hourly['Consumption_corrected'] + 1e-6

df_hourly['y'] = df_hourly['Consumption_corrected']

# Feature engineering
# Lag features (past 24 hours, one day back)
for lag in range(1, 25):
    df_hourly[f'lag_{lag}'] = df_hourly['y'].shift(lag)

# Rolling stats over past 24 hours
df_hourly['rolling_mean_24'] = df_hourly['y'].shift(1).rolling(24).mean()
df_hourly['rolling_std_24']  = df_hourly['y'].shift(1).rolling(24).std()

# Temperature lag features
df_hourly['temp_lag1'] = df_hourly['temperature'].shift(1)
df_hourly['temp_mean_24'] = df_hourly['temperature'].shift(1).rolling(24).mean()
df_hourly['temp_std_24']  = df_hourly['temperature'].shift(1).rolling(24).std()

# Time-based features
df_hourly['hour'] = df_hourly.index.hour
df_hourly['dayofweek'] = df_hourly.index.dayofweek
df_hourly['is_weekend'] = (df_hourly['dayofweek'] >= 5).astype(int)

# Drop missing values created by shifting/rolling
df_hourly = df_hourly.dropna()


# Train/Test split (last 30 days = 720 hours)
train = df_hourly.iloc[:-720]
test  = df_hourly.iloc[-720:]

X_train = train.drop(columns=['y', 'Consumption_corrected'])
y_train = train['y']
X_test = test.drop(columns=['y', 'Consumption_corrected'])
y_test = test['y']


# Train LightGBM model
lgb_model = lgb.LGBMRegressor(
    n_estimators=600,
    learning_rate=0.03,
    max_depth=10,
    num_leaves=31,
    subsample=0.8,
    colsample_bytree=0.8,
    random_state=42
)

lgb_model.fit(X_train, y_train)


# Forecast and evaluate
y_pred = lgb_model.predict(X_test)

mae = mean_absolute_error(y_test, y_pred)
rmse = np.sqrt(mean_squared_error(y_test, y_pred))
nrmse = rmse / (np.max(y_test) - np.min(y_test))
mase = mae / np.mean(np.abs(np.diff(y_train)))
r2 = r2_score(y_test, y_pred)


print(" LightGBM Forecast Evaluation – Last 30 Days (Hourly)")
print("-------------------------------------------")
print(f"MAE  : {mae:.3f}")
print(f"RMSE : {rmse:.3f}")
print(f"NRMSE (range) : {nrmse*100:.2f}%")
print(f"MASE : {mase:.2f}")
print(f"R²   : {r2:.3f}")



# Plot
plt.figure(figsize=(12,5))
plt.plot(y_test.index, y_test, label="Actual (Last 30 Days)", color="black", linewidth=1.6)
plt.plot(y_test.index, y_pred, label="Predicted (LightGBM)", color="orange", linewidth=1.8)
plt.title(f"LightGBM Forecast (Hourly Fuel Consumption)-Device {int(device_id)}")
plt.xlabel("Timestamp")
plt.ylabel("Fuel Consumption (Liters/hour)")
plt.legend()
plt.grid(True, linestyle="--", alpha=0.5)

# Date tick formatting
plt.gcf().autofmt_xdate(rotation=45)   # auto-rotate to avoid overlap
plt.tight_layout()


plt.tight_layout()
plt.show()


# Feature importance
# ---------------------------------------------------------------
feat_imp = pd.Series(lgb_model.feature_importances_, index=X_train.columns).sort_values(ascending=False)
plt.figure(figsize=(8,5))
sns.barplot(x=feat_imp[:10], y=feat_imp.index[:10], color='orange')
plt.title(f"Top 10 Feature Importances – LightGBM (Device {int(device_id)})")
plt.xlabel("Relative Importance")
plt.tight_layout()
plt.show()


XGBoost Forecast

In [ ]:
import seaborn as sns
from xgboost import XGBRegressor
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score

# Prepare hourly dataset
df_hourly = df_stl[['Consumption_corrected', 'temperature']].copy()

# Target variable
df_hourly['y'] = df_hourly['Consumption_corrected']


# Feature engineering
# Lag features (past 24 hours)
for lag in range(1, 25):
    df_hourly[f'lag_{lag}'] = df_hourly['y'].shift(lag)

# Rolling statistics (past 24 hours window)
df_hourly['rolling_mean_24'] = df_hourly['y'].shift(1).rolling(24).mean()
df_hourly['rolling_std_24']  = df_hourly['y'].shift(1).rolling(24).std()

# Temperature-based features
df_hourly['temp_lag1']  = df_hourly['temperature'].shift(1)
df_hourly['temp_mean24'] = df_hourly['temperature'].shift(1).rolling(24).mean()
df_hourly['temp_std24']  = df_hourly['temperature'].shift(1).rolling(24).std()

# Calendar/time-based features
df_hourly['hour'] = df_hourly.index.hour
df_hourly['dayofweek'] = df_hourly.index.dayofweek
df_hourly['is_weekend'] = (df_hourly['dayofweek'] >= 5).astype(int)

# Drop rows with NaNs from shifting/rolling
df_hourly = df_hourly.dropna()


# Train/test split (last 30 days = 720 hours)
train = df_hourly.iloc[:-720]
test  = df_hourly.iloc[-720:]

X_train = train.drop(columns=['y', 'Consumption_corrected'])
y_train = train['y']
X_test  = test.drop(columns=['y', 'Consumption_corrected'])
y_test  = test['y']


# Train XGBoost model
xgb_model = XGBRegressor(
    n_estimators=600,
    learning_rate=0.03,
    max_depth=8,
    subsample=0.8,
    colsample_bytree=0.8,
    random_state=42,
    reg_lambda=1.0
)

xgb_model.fit(X_train, y_train)


# Forecast and evaluate
y_pred = xgb_model.predict(X_test)

mae = mean_absolute_error(y_test, y_pred)
rmse = np.sqrt(mean_squared_error(y_test, y_pred))
nrmse = rmse / (np.max(y_test) - np.min(y_test))
mase = mae / np.mean(np.abs(np.diff(y_train)))
r2 = r2_score(y_test, y_pred)


print(" LightGBM Forecast Evaluation – Last 30 Days (Hourly)")
print("-------------------------------------------")
print(f"MAE  : {mae:.3f}")
print(f"RMSE : {rmse:.3f}")
print(f"NRMSE (range) : {nrmse*100:.2f}%")
print(f"MASE : {mase:.2f}")
print(f"R²   : {r2:.3f}")



# Plot actual vs predicted (Zoomed on last 30 days)
plt.figure(figsize=(12,5))
plt.plot(y_test.index, y_test, label="Actual (Last 30 Days)", color="black", linewidth=1.6)
plt.plot(y_test.index, y_pred, label="Predicted (XGBoost)", color="red", linewidth=2)
plt.title(f"XGBoost Forecast (Hourly Fuel Consumption)-Device {int(device_id)}", fontsize=13)
plt.xlabel("Timestamp")
plt.ylabel("Fuel Consumption (Liters/hour)")
plt.legend()
plt.grid(True, linestyle="--", alpha=0.5)
plt.gcf().autofmt_xdate(rotation=45)   # auto-rotate to avoid overlap
plt.tight_layout()
plt.tight_layout()
plt.show()


# Feature importance
# ---------------------------------------------------------------
feat_imp = pd.Series(xgb_model.feature_importances_, index=X_train.columns).sort_values(ascending=False)

plt.figure(figsize=(8,5))
sns.barplot(x=feat_imp[:10], y=feat_imp.index[:10], color='crimson')
plt.title(f"Top 10 Feature Importances – XGBoost (Device {int(device_id)})", fontsize=12)
plt.xlabel("Relative Importance")
plt.tight_layout()
plt.show()


Random Forest Forecast

In [ ]:
import seaborn as sns
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score


# Prepare hourly dataset
df_hourly = df_stl[['Consumption_corrected', 'temperature']].copy().dropna()
df_hourly['y'] = df_hourly['Consumption_corrected']

# Feature engineering
# Lag features (past 24 hours)
for lag in range(1, 25):
    df_hourly[f'lag_{lag}'] = df_hourly['y'].shift(lag)

# Rolling window features
df_hourly['rolling_mean_24'] = df_hourly['y'].shift(1).rolling(24).mean()
df_hourly['rolling_std_24']  = df_hourly['y'].shift(1).rolling(24).std()

# Temperature features
df_hourly['temp_lag1']   = df_hourly['temperature'].shift(1)
df_hourly['temp_mean24'] = df_hourly['temperature'].shift(1).rolling(24).mean()
df_hourly['temp_std24']  = df_hourly['temperature'].shift(1).rolling(24).std()

# Calendar/time features
df_hourly['hour']       = df_hourly.index.hour
df_hourly['dayofweek']  = df_hourly.index.dayofweek
df_hourly['is_weekend'] = (df_hourly['dayofweek'] >= 5).astype(int)

# Drop NaN rows created by lags/rolling windows
df_hourly = df_hourly.dropna()

# Train/Test split (last 30 days = 720 hours)
train = df_hourly.iloc[:-720]
test  = df_hourly.iloc[-720:]

X_train = train.drop(columns=['y', 'Consumption_corrected'])
y_train = train['y']
X_test  = test.drop(columns=['y', 'Consumption_corrected'])
y_test  = test['y']


# Train Random Forest model
rf_model = RandomForestRegressor(
    n_estimators=400,
    random_state=42,
    max_depth=15,
    min_samples_split=5,
    min_samples_leaf=3,
    n_jobs=-1
)
rf_model.fit(X_train, y_train)

# Forecast and evaluate
y_pred = rf_model.predict(X_test)

mae  = mean_absolute_error(y_test, y_pred)
rmse = np.sqrt(mean_squared_error(y_test, y_pred))
nrmse = rmse / (np.max(y_test) - np.min(y_test))
mase = mae / np.mean(np.abs(np.diff(y_train)))
r2   = r2_score(y_test, y_pred)


print(" Random Forest Forecast Evaluation – Last 30 Days (Hourly)")
print("-------------------------------------------")
print(f"MAE  : {mae:.4f}")
print(f"RMSE : {rmse:.4f}")
print(f"NRMSE (range): {nrmse*100:.2f}%")
print(f"MASE : {mase:.2f}")
print(f"R²   : {r2:.3f}")


# Plot actual vs predicted (Full 30 Days)
plt.figure(figsize=(12,5))
plt.plot(y_test.index, y_test, color='black', linewidth=1.5, label='Actual (Last 30 Days)')
plt.plot(y_test.index, y_pred, color='purple', linewidth=2, label='Predicted (Random Forest)')
plt.title(f"Random Forest Forecast (Hourly Fuel Consumption)-Device {int(device_id)}", fontsize=13)
plt.xlabel("Timestamp")
plt.ylabel("Fuel Consumption (Liters/hour)")
plt.legend()
plt.grid(True, linestyle="--", alpha=0.5)
plt.gcf().autofmt_xdate(rotation=45)   # auto-rotate to avoid overlap
plt.tight_layout()
plt.tight_layout()
plt.show()


# Feature importance
# ---------------------------------------------------------------
feat_imp = pd.Series(rf_model.feature_importances_, index=X_train.columns).sort_values(ascending=False)

plt.figure(figsize=(8,5))
sns.barplot(x=feat_imp[:10], y=feat_imp.index[:10], color='purple')
plt.title(f"Top 10 Feature Importances – Random Forest (Device {int(device_id)})")
plt.xlabel("Relative Importance")
plt.tight_layout()
plt.show()


LSTM Forecast

In [ ]:
import seaborn as sns
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout
from tensorflow.keras.callbacks import EarlyStopping

# Prepare hourly dataset
df_hourly = df_stl[['Consumption_corrected', 'temperature']].copy().dropna()
df_hourly['y'] = df_hourly['Consumption_corrected']


# Feature engineering
# Lag features (past 24 hours)
for lag in range(1, 25):
    df_hourly[f'lag_{lag}'] = df_hourly['y'].shift(lag)

# Rolling window stats (24-hour window)
df_hourly['rolling_mean_24'] = df_hourly['y'].shift(1).rolling(24).mean()
df_hourly['rolling_std_24']  = df_hourly['y'].shift(1).rolling(24).std()

# Temperature features
df_hourly['temp_lag1']   = df_hourly['temperature'].shift(1)
df_hourly['temp_mean24'] = df_hourly['temperature'].shift(1).rolling(24).mean()
df_hourly['temp_std24']  = df_hourly['temperature'].shift(1).rolling(24).std()

# Calendar features
df_hourly['hour']       = df_hourly.index.hour
df_hourly['dayofweek']  = df_hourly.index.dayofweek
df_hourly['is_weekend'] = (df_hourly['dayofweek'] >= 5).astype(int)

# Drop NaNs from lag/rolling feature creation
df_hourly = df_hourly.dropna()


# Train/Test split (last 30 days = 720 hours)
train = df_hourly.iloc[:-720]
test  = df_hourly.iloc[-720:]

X_train = train.drop(columns=['y', 'Consumption_corrected'])
y_train = train['y']
X_test  = test.drop(columns=['y', 'Consumption_corrected'])
y_test  = test['y']

# Scaling
scaler_X = MinMaxScaler()
scaler_y = MinMaxScaler()

X_train_scaled = scaler_X.fit_transform(X_train)
X_test_scaled  = scaler_X.transform(X_test)
y_train_scaled = scaler_y.fit_transform(y_train.values.reshape(-1, 1))
y_test_scaled  = scaler_y.transform(y_test.values.reshape(-1, 1))

# Reshape for LSTM: [samples, timesteps, features]
X_train_3d = X_train_scaled.reshape((X_train_scaled.shape[0], 1, X_train_scaled.shape[1]))
X_test_3d  = X_test_scaled.reshape((X_test_scaled.shape[0], 1, X_test_scaled.shape[1]))


# Build LSTM model
model = Sequential([
    LSTM(64, input_shape=(X_train_3d.shape[1], X_train_3d.shape[2]), return_sequences=False),
    Dropout(0.2),
    Dense(32, activation='relu'),
    Dense(1)
])
model.compile(optimizer='adam', loss='mse')

# Early stopping callback
early_stop = EarlyStopping(
    monitor='val_loss',
    patience=10,
    restore_best_weights=True,
    verbose=1
)

# Train the model
history = model.fit(
    X_train_3d, y_train_scaled,
    validation_data=(X_test_3d, y_test_scaled),
    epochs=200,
    batch_size=8,
    verbose=1,
    shuffle=False,
    callbacks=[early_stop]
)


# Forecast and evaluate
y_pred_scaled = model.predict(X_test_3d)
y_pred = scaler_y.inverse_transform(y_pred_scaled).flatten()
y_true = y_test.values.flatten()

mae  = mean_absolute_error(y_true, y_pred)
rmse = np.sqrt(mean_squared_error(y_true, y_pred))
nrmse = rmse / (np.max(y_true) - np.min(y_true))
mase = mae / np.mean(np.abs(np.diff(y_train)))
r2   = r2_score(y_true, y_pred)

print(" LSTM Forecast Evaluation – Last 30 Days (Hourly)")
print("-------------------------------------------")
print(f"MAE  : {mae:.4f}")
print(f"RMSE : {rmse:.4f}")
print(f"NRMSE (range): {nrmse*100:.2f}%")
print(f"MASE : {mase:.2f}")
print(f"R²   : {r2:.3f}")


# Training Loss Visualization
plt.figure(figsize=(7,4))
plt.plot(history.history['loss'], label='Train Loss', color='purple')
plt.plot(history.history['val_loss'], label='Validation Loss', color='orange')
plt.title("LSTM Training & Validation Loss (Hourly, Early Stopping)")
plt.xlabel("Epoch")
plt.ylabel("MSE Loss")
plt.legend()
plt.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()


# Actual vs Predicted (Full 30 Days)
plt.figure(figsize=(12,5))
plt.plot(y_test.index, y_true, label="Actual (Last 30 Days)", color="black", linewidth=1.5)
plt.plot(y_test.index, y_pred, label="Predicted (LSTM)", color="brown", linewidth=2)
plt.title(f"LSTM Forecast (Hourly Fuel Consumption)-Device {int(device_id)}", fontsize=13)
plt.xlabel("Timestamp")
plt.ylabel("Fuel Consumption (Liters/hour)")
plt.legend()
plt.grid(True, linestyle="--", alpha=0.5)
plt.gcf().autofmt_xdate(rotation=45)   # auto-rotate to avoid overlap
plt.tight_layout()
plt.tight_layout()
plt.show()


# STL Decomposition

STL Decomposition – Daily Temp-Corrected Fuel Consumption

In [ ]:
from statsmodels.tsa.seasonal import STL


# Prepare the daily dataset
# Aggregate hourly data to daily mean (or sum if you want total consumption)
df_daily_stl = df_stl[['Consumption_corrected']].resample('D').sum()
df_daily_stl = df_daily_stl.dropna()


# Perform STL decomposition
stl = STL(df_daily_stl['Consumption_corrected'], period=7, robust=True)
result = stl.fit()


# Visualize decomposition
plt.figure(figsize=(12, 8))
plt.suptitle("STL Decomposition – Daily Temperature-Corrected Fuel Consumption", fontsize=14, y=0.94)

plt.subplot(4, 1, 1)
plt.plot(df_daily_stl.index, df_daily_stl['Consumption_corrected'], color='black')
plt.title("Observed")
plt.grid(True, alpha=0.3)

plt.subplot(4, 1, 2)
plt.plot(df_daily_stl.index, result.trend, color='blue')
plt.title("Trend")
plt.grid(True, alpha=0.3)

plt.subplot(4, 1, 3)
plt.plot(df_daily_stl.index, result.seasonal, color='orange')
plt.title("Seasonal (Weekly Pattern)")
plt.grid(True, alpha=0.3)

plt.subplot(4, 1, 4)
plt.plot(df_daily_stl.index, result.resid, color='gray')
plt.title("Residual (Noise)")
plt.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()


# Combine components into a single DataFrame
df_stl_decomp = pd.DataFrame({
    'Observed': df_daily_stl['Consumption_corrected'],
    'Trend': result.trend,
    'Seasonal': result.seasonal,
    'Residual': result.resid
})

print("STL decomposition complete.")
print(df_stl_decomp.head())

Monthly Trend of Fuel Consumption (Based on STL Trend)

In [ ]:
# Aggregate to monthly totals using STL components
df_monthly_stl = (
    df_daily_result["Trend"]
    .resample("M")
    .sum()
    .to_frame("Monthly_Consumption")
)

df_monthly_stl["Year"] = df_monthly_stl.index.year
df_monthly_stl["MonthNum"] = df_monthly_stl.index.month

# Pivot: Month (1–12) vs Year
pivot = df_monthly_stl.pivot_table(
    index="MonthNum",
    columns="Year",
    values="Monthly_Consumption",
    aggfunc="sum"
).reindex(range(1, 13))  # ensure Jan–Dec order

# Plot with smoothing (NO extrapolation)
plt.figure(figsize=(10,6))

for year in pivot.columns:
    y = pivot[year]

    # keep only months with real data
    mask = y.notna()
    x_real = y.index[mask]
    y_real = y[mask]

    # 3-month centered rolling mean
    y_smooth = y_real.rolling(
        window=3,
        center=True,
        min_periods=1
    ).mean()

    plt.plot(
        x_real,
        y_smooth,
        linewidth=2.2,
        label=str(year)
    )

# Formatting

plt.title(f"Monthly Trend of Fuel Consumption (Based on STL Trend)-Device {int(device_id)}")
plt.xlabel("Month")
plt.ylabel("Fuel Consumption (Liters)")

plt.xticks(
    range(1, 13),
    ["Jan","Feb","Mar","Apr","May","Jun",
     "Jul","Aug","Sep","Oct","Nov","Dec"]
)

plt.grid(True, alpha=0.25)
plt.legend(title="Year", loc="upper right")
plt.tight_layout()
plt.show()


Average Hourly Fuel Consumption (Daily Cycle)

In [ ]:
df_stl["hour"] = df_stl.index.hour
df_stl.groupby("hour")["Consumption"].mean().plot(title=f"Average Hourly Fuel Consumption (Daily Cycle)-Device {int(device_id)}")

Total Daily Fuel Use by Weekday

In [ ]:
# Create daily totals for total daily fuel use
df_daily = df_stl["Consumption"].resample("D").sum()

# Add weekday names to daily data
df_daily = df_daily.to_frame(name="Daily_Total")
df_daily["weekday_name"] = df_daily.index.day_name()

# Define weekday order for proper x-axis order
order = ["Monday", "Tuesday", "Wednesday", "Thursday", "Friday", "Saturday", "Sunday"]

# Plot both daily and weekly patterns
fig, axes = plt.subplots(1, 2, figsize=(12, 4))

# Left: Daily (hourly pattern)
df_stl.groupby("hour")["Consumption"].mean().plot(ax=axes[0], color="steelblue")
axes[0].set_title(f"Average Hourly Fuel Consumption (Daily Cycle)-Device {int(device_id)}")
axes[0].set_xlabel("Hour of Day")
axes[0].set_ylabel("Liters per Hour")
axes[0].grid(True)

# Right: Weekly (total daily fuel use)
df_daily.groupby("weekday_name")["Daily_Total"].mean().reindex(order).plot(
    kind="bar", ax=axes[1], color="teal"
)
axes[1].set_title(f"Total Daily Fuel Use by Weekday-Device {int(device_id)}")
axes[1].set_xlabel("Day of Week")
axes[1].set_ylabel("Total Fuel Used (Liters per Day)")
axes[1].grid(True)

plt.tight_layout()
plt.show()




Average Hourly Fuel Consumption by Day of Week

In [ ]:
# Ensure datetime index
df_stl = df_stl.copy()
df_stl.index = pd.to_datetime(df_stl.index)

# Extract hour and weekday
df_stl["hour"] = df_stl.index.hour
df_stl["weekday"] = df_stl.index.day_name()

# Compute average consumption per (day, hour)
pivot = (
    df_stl.groupby(["weekday", "hour"])["Consumption"]
    .mean()
    .unstack(level=1)
)

# Reorder days of the week (so Monday starts first)
days_order = ["Monday", "Tuesday", "Wednesday", "Thursday", "Friday", "Saturday", "Sunday"]
pivot = pivot.reindex(days_order)

# PLOT
plt.figure(figsize=(12, 6))
sns.heatmap(
    pivot,
    cmap="YlOrRd",
    linewidths=0.3,
    linecolor="gray",
    cbar_kws={'label': 'Avg Fuel Consumption (L/h)'}
)
plt.title(f"Average Hourly Fuel Consumption by Day of Week – Device {int(device_id)}")
plt.xlabel("Hour of Day")
plt.ylabel("Day of Week")
plt.tight_layout()
plt.show()



Anomaly Detection extracted from residual component of STL

In [ ]:
# Prepare both residual signals ===
resid_clean = df_stl_decomp["Residual"]              # From STL after cleaning
resid_raw = raw_resid.reindex_like(resid_clean)        # Align index with cleaned version

# Compute anomaly thresholds (using ±3σ rule)
mean_clean, std_clean = resid_clean.mean(), resid_clean.std()
mean_raw, std_raw = resid_raw.mean(), resid_raw.std()

upper_clean, lower_clean = mean_clean + 3 * std_clean, mean_clean - 3 * std_clean
upper_raw, lower_raw = mean_raw + 3 * std_raw, mean_raw - 3 * std_raw

# Flag anomalies
anom_clean = resid_clean[(resid_clean > upper_clean) | (resid_clean < lower_clean)]
anom_raw = resid_raw[(resid_raw > upper_raw) | (resid_raw < lower_raw)]

print(f"Detected anomalies (raw residuals): {len(anom_raw)}")
print(f"Detected anomalies (cleaned residuals): {len(anom_clean)}")

# Plot comparison
fig, axes = plt.subplots(2, 1, figsize=(12, 8), sharex=True)

# Raw residuals
axes[0].plot(resid_raw.index, resid_raw, color="gray", label="Raw Residuals")
axes[0].axhline(upper_raw, color="red", linestyle="--", label="+3σ Threshold")
axes[0].axhline(lower_raw, color="red", linestyle="--")
axes[0].scatter(anom_raw.index, anom_raw, color="orange", s=40, label="Anomalies")
axes[0].set_title("Anomaly Detection Using Raw Residuals (Before Cleaning)")
axes[0].set_ylabel("Residual (Liters/day)")
axes[0].legend()
axes[0].grid(True, alpha=0.4)

# Cleaned residuals
axes[1].plot(resid_clean.index, resid_clean, color="gray", label="Cleaned Residuals")
axes[1].axhline(upper_clean, color="red", linestyle="--", label="+3σ Threshold")
axes[1].axhline(lower_clean, color="red", linestyle="--")
axes[1].scatter(anom_clean.index, anom_clean, color="orange", s=40, label="Anomalies")
axes[1].set_title("Anomaly Detection Using Cleaned STL Residuals (After IQR-Based Cleaning)")
axes[1].set_xlabel("Date")
axes[1].set_ylabel("Residual (Liters/day)")
axes[1].legend()
axes[1].grid(True, alpha=0.4)

plt.tight_layout()
plt.show()


Seasonality analysis of fuel consumption

In [ ]:
# Set the style
sns.set_style("white")

# Prepare monthly summary
df_monthly = df_daily_result.copy()
df_monthly["Year"] = df_monthly.index.year
df_monthly["Month"] = df_monthly.index.month

# Compute total or mean fuel use per month/year
monthly_summary = (
    df_monthly.groupby(["Year", "Month"])["Seasonal"]
    .mean()
    .reset_index()
)

# Create abbreviated month names
month_names = {1: "Jan", 2: "Feb", 3: "Mar", 4: "Apr", 5: "May", 6: "Jun",
               7: "Jul", 8: "Aug", 9: "Sep", 10: "Oct", 11: "Nov", 12: "Dec"}
monthly_summary["Month_Name"] = monthly_summary["Month"].map(month_names)

# Order for faceting
monthly_summary["Month_Name"] = pd.Categorical(
    monthly_summary["Month_Name"],
    categories=list(month_names.values()),
    ordered=True
)

# Create facet grid (seasonality panel)
fig = plt.figure(figsize=(16, 3.5))

g = sns.FacetGrid(
    monthly_summary,
    col="Month_Name",
    col_wrap=12,
    height=3,
    aspect=0.6,
    sharey=True,
    gridspec_kws={"hspace": 0.05, "wspace": 0.05}
)

# Plot the line and mean line
g.map_dataframe(sns.lineplot, x="Year", y="Seasonal", color="black", linewidth=1.2)
g.map_dataframe(lambda data, **kws: plt.axhline(
    data["Seasonal"].mean(),
    color="blue",
    linewidth=2.5,
    alpha=0.9
))

# Formatting
g.set_titles("{col_name}", fontsize=10, fontweight='bold', pad=8)

# Customize each axis
for i, ax in enumerate(g.axes.flat):
    # Set gray background
    ax.set_facecolor('#e8e8e8')

    # White gridlines
    ax.grid(True, axis='both', linestyle='-', linewidth=0.8, color='white', alpha=0.9)

    # Rotate x-axis labels to vertical
    ax.tick_params(axis='x', rotation=90, labelsize=7)
    ax.tick_params(axis='y', labelsize=8)

    # Format x-axis to show years properly
    ax.xaxis.set_major_locator(plt.MaxNLocator(integer=True, nbins=8))

    # Remove spines
    for spine in ax.spines.values():
        spine.set_visible(False)

    # Remove x-axis label
    ax.set_xlabel("")

    # Hide y-axis labels for all except the first subplot
    if i == 0:
        ax.tick_params(axis='y', which='both', length=4, labelsize=8)
        ax.set_ylabel("Fuel Consumption (Liters)", fontsize=9)
    else:
        ax.tick_params(axis='y', which='both', left=False, labelleft=False)
        ax.set_ylabel("")

# Adjust spacing to make room for bottom labels
plt.subplots_adjust(left=0.05, right=0.98, top=0.92, bottom=0.18)

# Add x-axis label at bottom
g.fig.suptitle("month",
               fontsize=11, y=0.01, fontweight='normal')

plt.show()